# Ollama + OpenAI + Python

## 1. Model adını belirtin

Eğer "phi3:mini" dışında bir model indirdiyseniz, aşağıdaki hücredeki değeri değiştirin. Bu değişken, not defteri boyunca kodda kullanılacaktır.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Open AI istemcisini kurun

Genellikle OpenAI istemcisi, büyük dil modelleriyle etkileşim kurmak için OpenAI.com veya Azure OpenAI ile kullanılır.  
Ancak, Ollama ile de kullanılabilir çünkü Ollama, "http://localhost:11434/v1" adresinde OpenAI uyumlu bir uç nokta sağlar.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Bir sohbet tamamlaması oluşturun

Artık OpenAI SDK'sını kullanarak bir konuşma için yanıt oluşturabiliriz. Bu istek, kediler hakkında bir haiku oluşturmalıdır:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. İstek mühendisliği

Dil modeline gönderilen ilk mesaj "sistem mesajı" veya "sistem isteği" olarak adlandırılır ve model için genel talimatları belirler.  
Kendi özel sistem isteğinizi sağlayarak bir dil modelini farklı bir şekilde çıktı üretmesi için yönlendirebilirsiniz.  
Aşağıdaki `SYSTEM_MESSAGE` bölümünü, favori film/TV karakteriniz gibi cevap verecek şekilde değiştirin veya diğer sistem istekleri için [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) adresinden ilham alın.  

Sistem mesajını özelleştirdikten sonra, `USER_MESSAGE` bölümüne ilk kullanıcı sorusunu ekleyin.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Az örnekli örnekler

Bir dil modelini yönlendirmenin başka bir yolu, "az örnekli" bir yaklaşım kullanmaktır; bu, modelin nasıl yanıt vermesi gerektiğini gösteren bir dizi örnek soru/cevap sağlamaktır.

Aşağıdaki örnek, bir dil modelini bir öğretim asistanı gibi davranmaya yönlendirmeye çalışır. Bunun için bir öğretim asistanının verebileceği birkaç soru ve cevap örneği sunulur ve ardından modele bir öğrencinin sorabileceği bir soru iletilir.

Önce bunu deneyin, ardından `SYSTEM_MESSAGE`, `EXAMPLES` ve `USER_MESSAGE` öğelerini yeni bir senaryo için değiştirin.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Bilgiye Dayalı Üretim

RAG (Bilgiye Dayalı Üretim), bir dil modelinin belirli bir alanla ilgili soruları doğru bir şekilde yanıtlamasını sağlamak için önce bir bilgi kaynağından ilgili bilgileri alıp ardından bu bilgilere dayanarak bir yanıt oluşturma tekniğidir.

Hibrit arabalarla ilgili veriler içeren bir yerel CSV dosyası sağladık. Aşağıdaki kod, CSV dosyasını okur, kullanıcı sorusuna uygun eşleşmeleri arar ve bulunan bilgilere dayanarak bir yanıt oluşturur. Bunun önceki örneklerden daha uzun süreceğini unutmayın, çünkü modele daha fazla veri gönderilir. Eğer yanıtın hala verilere dayalı olmadığını fark ederseniz, sistem mühendisliğini deneyebilir veya başka modelleri kullanabilirsiniz. Genel olarak, RAG daha büyük modellerle veya SLM'lerin ince ayar yapılmış versiyonlarıyla daha etkili olur.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul etmiyoruz.
